In [3]:
import numpy as np
import pandas as pd
from numpy import unique
from numpy import where

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

import matplotlib.pyplot as plt

# from pipelinehelper.pipelinehelper import PipelineHelper

In [4]:
# Load data
data = pd.read_csv("data/data.csv",  header=None)
target = pd.read_csv("data/id.csv", header=None)

In [5]:
#print(data.head)
print("INFO data")
print(data.info(verbose=True))
print("INFO target")
print(target.info(verbose=True))

INFO data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 128 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64


In [6]:
# Data train into numpy
X = np.array(data.values)
print("INFO data train ndim ", X.ndim)
print("INFO data train shape ", X.shape)
# Data target into numpy
y = np.array(target.values)
le.fit([0, 1, 2, 3, 4, 5])
le.transform(y)
print("INFO data target ndim ", y.ndim)
print("INFO data target shape ", y.shape)
print("INFO data target example", y[:-10])

INFO data train ndim  2
INFO data train shape  (2450, 128)
INFO data target ndim  2
INFO data target shape  (2450, 1)
INFO data target example [[0]
 [0]
 [0]
 ...
 [5]
 [5]
 [5]]


In [7]:
# Get indexes of all labels

samet_index = data[target[0]==0].index.tolist()
meyra_index = data[target[0]==1].index.tolist()
bekir_index = data[target[0]==2].index.tolist()
samir_index = data[target[0]==3].index.tolist()
berat_index = data[target[0]==4].index.tolist()
ramazan_index = data[target[0]==5].index.tolist()

# Show first 10 indexes
print("Bekir indexes ", bekir_index[:10])
print("Berat indexes ", berat_index[:10])
print("Meyra indexes ", meyra_index[:10])
print("Ramazan indexes ", ramazan_index[:10])
print("Samet indexes ", samet_index[:10])
print("Samir indexes ", samir_index[:10])

Bekir indexes  [934, 935, 936, 937, 938, 939, 940, 941, 942, 943]
Berat indexes  [1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725]
Meyra indexes  [479, 480, 481, 482, 483, 484, 485, 486, 487, 488]
Ramazan indexes  [2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258]
Samet indexes  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Samir indexes  [1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394]


In [8]:
# Get all indexes together
all_indexes = np.concatenate([bekir_index[:50],
                                berat_index[:50],
                                meyra_index[:50],
                                ramazan_index[:50],
                                samet_index[:50],
                                samir_index[:50]])
# all_indexes = np.vstack(all_indexes,
print(all_indexes)

[ 934  935  936  937  938  939  940  941  942  943  944  945  946  947
  948  949  950  951  952  953  954  955  956  957  958  959  960  961
  962  963  964  965  966  967  968  969  970  971  972  973  974  975
  976  977  978  979  980  981  982  983 1716 1717 1718 1719 1720 1721
 1722 1723 1724 1725 1726 1727 1728 1729 1730 1731 1732 1733 1734 1735
 1736 1737 1738 1739 1740 1741 1742 1743 1744 1745 1746 1747 1748 1749
 1750 1751 1752 1753 1754 1755 1756 1757 1758 1759 1760 1761 1762 1763
 1764 1765  479  480  481  482  483  484  485  486  487  488  489  490
  491  492  493  494  495  496  497  498  499  500  501  502  503  504
  505  506  507  508  509  510  511  512  513  514  515  516  517  518
  519  520  521  522  523  524  525  526  527  528 2249 2250 2251 2252
 2253 2254 2255 2256 2257 2258 2259 2260 2261 2262 2263 2264 2265 2266
 2267 2268 2269 2270 2271 2272 2273 2274 2275 2276 2277 2278 2279 2280
 2281 2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294
 2295 

In [9]:
# get data and labels of all indexes
x_ten = data.loc[ all_indexes, : ]
y_ten = target.loc[ all_indexes , : ]

print("x_ten", x_ten)
print("y_ten", y)

x_ten            0         1         2         3         4         5         6    \
934  -0.141236  0.182262  0.056664  0.152511  0.083062  0.023147 -0.092564   
935  -0.172426  0.129670 -0.045568  0.080295  0.105105 -0.016185 -0.154171   
936  -0.128303  0.167794 -0.004384  0.085313  0.171333  0.047267 -0.121825   
937  -0.137400  0.129731 -0.022547  0.082919  0.111654  0.082956 -0.093362   
938  -0.110589  0.173985  0.081496  0.163161  0.123259  0.008574 -0.142290   
...        ...       ...       ...       ...       ...       ...       ...   
1430  0.000243  0.055220  0.035373  0.001815 -0.046349  0.119317 -0.226786   
1431 -0.036954  0.077120  0.121743 -0.033962 -0.057352  0.107758 -0.174992   
1432 -0.028559  0.048405 -0.016701  0.027891 -0.108294  0.097568 -0.193534   
1433 -0.026298  0.071113  0.031832 -0.020578 -0.049751  0.078993 -0.188323   
1434 -0.028949  0.065760  0.042407  0.012907 -0.080943  0.081421 -0.170090   

           7         8         9    ...       118       1

In [30]:
# Real test data
X_train, X_test, y_train, y_test = train_test_split(x_ten, y_ten, test_size=0.7, random_state=42)

In [42]:
# Supervised SVC parameter grid search
n_samples = len(X_train)
print("Train data lenght ",n_samples)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['poly'], 'degree': [1, 3, 5, 7]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

Train data lenght  180
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.032 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.032 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.032 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.032 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.982 (+/-0.018) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.032 (+/-0.000) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.982 (+/-0.018) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.982 (+/-0.018) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.982 (+/-0.018) for {'C': 1, 'kernel': 'linear'}
0.986 (+/-0.023) for {'C': 10, 'kernel': 'linear'}
0.986 (+/-0.023) for {'C': 100, 'kernel': 'linear'}
0.986 (+/-0.023) for {'C': 1000, 'kernel': 'linear'}
0.982 (+/-0.018) for {'degree': 1, 'kernel': 'poly'}
0.982 (+/-0.019) for {'degree': 3, 'kernel': '

In [10]:
# Normalize data
scale_norm = preprocessing.StandardScaler(with_mean=True).fit(X_train)
X_train_norm = scale_norm.transform(X_train)
X_test_norm = scale_norm.transform(X_test)

In [55]:
# Supervised Normal Bayes parameter grid search

n_samples = len(X_train_norm)
n_samples_test = len(X_test_norm)
print("Train data lenght ",n_samples)
print("Test data lenght ",n_samples_test)

# Set the parameters by cross-validation
tuned_parameters = [{ 'var_smoothing': [1000e-1, 100e-1, 10e-1, 1e-1, 1e-2, 1e-3, 1e-4]}
                    ]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(GaussianNB(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train_norm, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test_norm)
    print(classification_report(y_true, y_pred))
    print()
    y_true = np.array(y_true).flatten()
    print("Label y_true {}" .format(y_true))
    print("Label y_pred {}" .format(y_pred))
    print()
    print("Number of mislabeled points out of a total %d points: %d"
     % (X_test.shape[0], (y_true != y_pred).sum()))
    print()
    print()

Train data lenght  180
Test data lenght  120
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'var_smoothing': 100.0}

Grid scores on development set:

0.992 (+/-0.020) for {'var_smoothing': 100.0}
0.992 (+/-0.020) for {'var_smoothing': 10.0}
0.982 (+/-0.018) for {'var_smoothing': 1.0}
0.982 (+/-0.019) for {'var_smoothing': 0.1}
0.982 (+/-0.019) for {'var_smoothing': 0.01}
0.982 (+/-0.019) for {'var_smoothing': 0.001}
0.982 (+/-0.019) for {'var_smoothing': 0.0001}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00        19
           2       1.00      0.95      0.97        20
           3       1.00      1.00      1.00        15
           4       0.96      1.00      0.98        24
           5       1.00      

In [31]:
# Supervised KNN parameter grid search

n_samples = len(X_train)
n_samples_test = len(X_test)
print("Train data lenght ",n_samples)
print("Test data lenght ",n_samples_test)

# Set classifier by cross-validation
# pipe = Pipeline([
#     ('classifier', PipelineHelper([
#         ('knn', KNeighborsClassifier()),
#     ])),
# ])

# Set the parameters by cross-validation
tuned_parameters = [{ 'n_neighbors': [6],
                    'algorithm': ['brute', 'kd_tree'],
                    'p': [1,2]}]

scores = ['precision', 'recall']

for score in scores:
    print("############ Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(KNeighborsClassifier(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
    y_true = np.array(y_true).flatten()
    print("Label y_true {}" .format(y_true))
    print("Label y_pred {}" .format(y_pred))
    print()
    print("Number of mislabeled points out of a total %d points: %d"
     % (X_test.shape[0], (y_true != y_pred).sum()))
    print()
    print()
    y_true, y_pred_proba = y_test, clf.predict_proba(X_test)
    y_pred_proba = np.argmax(y_pred_proba, axis=1)
    print(classification_report(y_true, y_pred_proba))
    print()
    y_true = np.array(y_true).flatten()
    print("Label y_true {}" .format(y_true))
    print("Label y_pred_proba {}" .format(y_pred_proba))
    print()
    print("Number of mislabeled points out of a total %d points: %d"
     % (X_test.shape[0], (y_true != y_pred_proba).sum()))
    print()

Train data lenght  90
Test data lenght  210
############ Tuning hyper-parameters for precision

Best parameters set found on development set:

{'algorithm': 'brute', 'n_neighbors': 6, 'p': 2}

Grid scores on development set:

0.983 (+/-0.041) for {'algorithm': 'brute', 'n_neighbors': 6, 'p': 1}
0.985 (+/-0.037) for {'algorithm': 'brute', 'n_neighbors': 6, 'p': 2}
0.983 (+/-0.041) for {'algorithm': 'kd_tree', 'n_neighbors': 6, 'p': 1}
0.985 (+/-0.037) for {'algorithm': 'kd_tree', 'n_neighbors': 6, 'p': 2}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       0.95      0.97      0.96        36
           2       1.00      0.97      0.99        34
           3       0.97      0.94      0.95        32
           4       0.97      1.00      0.99        35
           5       1.00     